In [151]:
import numpy as np
import pandas as pd
import datetime
import warnings
import geopy
import pickle

In [152]:
with open('../src/data/keys/authkeys', 'rb') as authkeys:
    GOOGLE_AUTH = pickle.load(authkeys)
geolocator = geopy.geocoders.GoogleV3(api_key=GOOGLE_AUTH)
geopy.geocoders.options.default_timeout = 1

In [154]:
hdb=pd.read_csv('../data/interim/hdbaddresses.csv')
hdb.rename(columns = {'0':'address'}, inplace=True)
hdb

,address
0,Ang Mo Kio Avenue 4 Block 172 Singapore
1,Ang Mo Kio Avenue 8 Block 510 Singapore
2,Ang Mo Kio Avenue 4 Block 610 Singapore
3,Ang Mo Kio Avenue 10 Block 474 Singapore
4,Ang Mo Kio Avenue 5 Block 604 Singapore
...,...
9123,Edgedale Plains Block 122A Singapore
9124,Punggol Drive Block 677D Singapore
9125,Compassvale Drive Block 216B Singapore
9126,Joo Seng Road Block 9 Singapore


In [66]:
jcs=pd.read_json('../data/interim/jcs.json')
jcs = jcs.replace(r'\n',' ', regex=True) 
jcs.head(10)

,area,school_name,address,schooltype
0,Ang Mo Kio,Anderson Serangoon Junior College,4500 Ang Mo Kio Avenue 6,Government
1,Queenstown,Anglo-Chinese Junior College,25 Dover Close East,Government-aided
2,Queenstown,Anglo-Chinese IB Junior College,121 Dover Road,Independent
3,Novena,Catholic Junior College,129 Whitley Road,Government-aided
4,Kallang,Dunman High School,10 Tanjong Rhu Road,Autonomous
5,Queenstown,Eunoia Junior College,53 Mount Sinai Road (holding site),Government
6,Bukit Timah,Hwa Chong Junior College,661 Bukit Timah Road,Independent
7,Choa Chu Kang,Jurong Pioneer Junior College,21 Teck Whye Walk,Government
8,Serangoon,Nanyang Junior College,128 Serangoon Avenue 3,Government-aided
9,Bukit Timah,National Junior College,37 Hillcrest Road,Government


In [67]:
postsec=pd.read_json('../data/interim/postsec.json')
postsec.head(10)

,school_name,studentpop
0,Singapore Polytechnic,"14,337"
1,Ngee Ann Polytechnic,"14,543"
2,Temasek Polytechnic,"14,248"
3,Nanyang Polytechnic,"14,715"
4,Republic Polytechnic,"13,305"
5,Nanyang Academy of Fine Arts,"2,300"
6,LASALLE College of the Arts,"2,500\n"
7,Orita Sinclair School of Design & Music,\n


In [38]:
prisch=pd.read_json('../data/interim/prisch.json')
prisch.head(10)

,area,school_name,schooltype,gendertype
0,Woodlands,Admiralty Primary School,Government,Mixed
1,Yishun,Ahmad Ibrahim Primary School,Government,Mixed
2,Bishan,Ai Tong School,"Government-aided,",Mixed
3,Bukit Merah,Alexandra Primary School,Government,Mixed
4,Sengkang,Anchor Green Primary School,Government,Mixed
5,Ang Mo Kio,Anderson Primary School,Government,Mixed
6,Central,Anglo-Chinese School (Junior),Government-aided,Boys
7,Bukit Timah,Anglo-Chinese School (Primary),Government-aided,Boys
8,Tampines,Angsana Primary School,Government,Mixed
9,Ang Mo Kio,Ang Mo Kio Primary School,Government,Mixed


In [40]:
secsch=pd.read_json('../data/interim/secsch.json')
secsch.head(10)

,area,school_name,schooltype
0,Woodlands,Admiralty Secondary School,Government
1,Yishun,Ahmad Ibrahim Secondary School,Government
2,Ang Mo Kio,Anderson Secondary School,Government
3,Bedok,Anglican High School,Government-aided
4,Newton,Anglo-Chinese School (Barker Road),Government-aided
5,Dover,Anglo-Chinese School (Independent),Independent
6,Ang Mo Kio,Ang Mo Kio Secondary School,Government
7,Bukit Panjang,Assumption English School,Government-aided
8,Toa Payoh,Bartley Secondary School,Government
9,Toa Payoh,Beatty Secondary School,Government


In [41]:
unis=pd.read_json('../data/interim/unis.json')
unis.head(10)

,school_name,abbrev,studentpop,schooltype,founded
0,National University of Singapore,NUS,"42,578",comprehensive research intensive university\n,1905
1,Nanyang Technological University,NTU,"32,015",comprehensive research intensive university\n,1981
2,Singapore Management University,SMU,"8,656",specialised research intensive university\n,2000
3,Singapore University of Technology and Design,SUTD,"1,341",specialised research intensive university\n,2009
4,Singapore Institute of Technology,SIT,"over 6,000",specialised university of applied learning\n,2009
5,Singapore University of Social Sciences,SUSS,"14,841",specialised university of applied and lifelong...,2005


In [130]:
mrt=pd.read_csv('../data/interim/mrtstations.csv')
mrt['stations'] = mrt['stations'].astype(str) + ' MRT'
mrt = mrt.drop(columns = ['Unnamed: 1'])
mrt.head()

,stations
0,Jurong East MRT
1,Bukit Batok MRT
2,Bukit Gombak MRT
3,Choa Chu Kang MRT
4,Yew Tee MRT


In [131]:
malls=pd.read_json('../data/interim/malls.json')
malls = pd.DataFrame(malls['malls'][0])
malls.rename(columns = {0:'mall'}, inplace=True)
malls

,mall
0,100 AM
1,313@Somerset
2,Aperia
3,Balestier Hill Shopping Centre
4,Bugis Cube
...,...
170,Gek Poh Shopping Centre
171,Rochester Mall
172,Taman Jurong Shopping Centre
173,West Coast Plaza


In [157]:
df = pd.concat([hdb['address'].astype(str),
                (jcs['school_name'].astype(str) + ' Singapore'),
                (postsec['school_name'].astype(str) + ' Singapore'),
                (prisch['school_name'].astype(str) + ' Singapore'),
                (secsch['school_name'].astype(str) + ' Singapore'),
                (unis['school_name'].astype(str) + ' Singapore'),
                (mrt['stations'].astype(str) + ' Singapore'),
                (malls['mall'].astype(str) + ' Singapore')], ignore_index=True)
df = df.to_frame()
df.rename(columns = {0:'geoquery'}, inplace=True)
df['geoquery'] = df['geoquery'].astype(str)
df.head()

,geoquery
0,Ang Mo Kio Avenue 4 Block 172 Singapore
1,Ang Mo Kio Avenue 8 Block 510 Singapore
2,Ang Mo Kio Avenue 4 Block 610 Singapore
3,Ang Mo Kio Avenue 10 Block 474 Singapore
4,Ang Mo Kio Avenue 5 Block 604 Singapore


In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9795 entries, 0 to 9794
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   geoquery  9795 non-null   object
dtypes: object(1)
memory usage: 76.6+ KB


In [159]:
df

,geoquery
0,Ang Mo Kio Avenue 4 Block 172 Singapore
1,Ang Mo Kio Avenue 8 Block 510 Singapore
2,Ang Mo Kio Avenue 4 Block 610 Singapore
3,Ang Mo Kio Avenue 10 Block 474 Singapore
4,Ang Mo Kio Avenue 5 Block 604 Singapore
...,...
9790,Gek Poh Shopping Centre Singapore
9791,Rochester Mall Singapore
9792,Taman Jurong Shopping Centre Singapore
9793,West Coast Plaza Singapore


In [160]:
# Perform the address queries
df['loc'] = df['geoquery'].apply(geolocator.geocode)

In [161]:
df['point']= df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat', 'lon', 'altitude']] = pd.DataFrame(df['point'].to_list(), index=df.index)
df.to_csv('../data/interim/geocodes', index=False)
with pd.option_context('display.max_colwidth', None):
  display(df)

,geoquery,loc,point,lat,lon,altitude
0,Ang Mo Kio Avenue 4 Block 172 Singapore,"(172 Ang Mo Kio Ave 4, Singapore 560172, (1.3746384, 103.8365012))","(1.3746384, 103.8365012, 0.0)",1.374638,103.836501,0.0
1,Ang Mo Kio Avenue 8 Block 510 Singapore,"(510 Ang Mo Kio Ave 8, Block 510, Singapore 560510, (1.3734149, 103.8492068))","(1.3734149, 103.8492068, 0.0)",1.373415,103.849207,0.0
2,Ang Mo Kio Avenue 4 Block 610 Singapore,"(610 Ang Mo Kio Ave 4, Block 610, Singapore 560610, (1.3798478, 103.8394238))","(1.3798478, 103.8394238, 0.0)",1.379848,103.839424,0.0
3,Ang Mo Kio Avenue 10 Block 474 Singapore,"(474 Ang Mo Kio Ave 10, Block 474, Singapore 560474, (1.3628141, 103.8579939))","(1.3628141, 103.8579939, 0.0)",1.362814,103.857994,0.0
4,Ang Mo Kio Avenue 5 Block 604 Singapore,"(604 Ang Mo Kio Ave 5, Block 604, Singapore 560604, (1.3800876, 103.8360724))","(1.3800876, 103.8360724, 0.0)",1.380088,103.836072,0.0
...,...,...,...,...,...,...
9790,Gek Poh Shopping Centre Singapore,"(Gek Poh Shopping Centre, 762 Jurong West Street 75, Singapore 640762, (1.3488852, 103.6976313))","(1.3488852, 103.6976313, 0.0)",1.348885,103.697631,0.0
9791,Rochester Mall Singapore,"(35 Rochester Dr, Singapore 138639, (1.3056024, 103.788206))","(1.3056024, 103.788206, 0.0)",1.305602,103.788206,0.0
9792,Taman Jurong Shopping Centre Singapore,"(399 Yung Sheng Rd, Singapore 610399, (1.3342811, 103.7203739))","(1.3342811, 103.7203739, 0.0)",1.334281,103.720374,0.0
9793,West Coast Plaza Singapore,"(154 West Coast Rd, Singapore 127371, (1.303632, 103.766452))","(1.303632, 103.766452, 0.0)",1.303632,103.766452,0.0


In [165]:
df.to_csv('../data/interim/geocodes.csv', index=False)


In [166]:
test = pd.read_csv('../data/interim/geocodes.csv')
test.head()

,geoquery,loc,point,lat,lon,altitude
0,Ang Mo Kio Avenue 4 Block 172 Singapore,"172 Ang Mo Kio Ave 4, Singapore 560172","(1.3746384, 103.8365012, 0.0)",1.374638,103.836501,0.0
1,Ang Mo Kio Avenue 8 Block 510 Singapore,"510 Ang Mo Kio Ave 8, Block 510, Singapore 560510","(1.3734149, 103.8492068, 0.0)",1.373415,103.849207,0.0
2,Ang Mo Kio Avenue 4 Block 610 Singapore,"610 Ang Mo Kio Ave 4, Block 610, Singapore 560610","(1.3798478, 103.8394238, 0.0)",1.379848,103.839424,0.0
3,Ang Mo Kio Avenue 10 Block 474 Singapore,"474 Ang Mo Kio Ave 10, Block 474, Singapore 56...","(1.3628141, 103.8579939, 0.0)",1.362814,103.857994,0.0
4,Ang Mo Kio Avenue 5 Block 604 Singapore,"604 Ang Mo Kio Ave 5, Block 604, Singapore 560604","(1.3800876, 103.8360724, 0.0)",1.380088,103.836072,0.0


In [ ]:
import requests
import json
from requests.exceptions import HTTPError

In [ ]:
def distancecall(df):
    

In [17]:
df[pd.isnull(df).any(axis=1)]

,address,loc,point,lat,lon,altitude
6077,"Kallang/Whampoa, Jln Bahagia Block Number 39 S...",NaN,NaN,NaN,NaN,NaN
8066,"Kallang/Whampoa, Jln Bahagia Block Number 55 S...",NaN,NaN,NaN,NaN,NaN
8556,"Kallang/Whampoa, Jln Bahagia Block Number 52 S...",NaN,NaN,NaN,NaN,NaN
8923,"Kallang/Whampoa, Jln Bahagia Block Number 47 S...",NaN,NaN,NaN,NaN,NaN


In [26]:
df[df['address'].str.contains(' Ave ')]

,address,loc,point,lat,lon,altitude
0,"Ang Mo Kio, Ang Mo Kio Ave 4 Block Number 172 ...","172 Ang Mo Kio Ave 4, Singapore 560172","(1.3747742, 103.8364795, 0.0)",1.374774,103.836479,0.0
1,"Ang Mo Kio, Ang Mo Kio Ave 8 Block Number 510 ...","510 Ang Mo Kio Ave 8, Singapore 560510","(1.373372, 103.848988, 0.0)",1.373372,103.848988,0.0
2,"Ang Mo Kio, Ang Mo Kio Ave 4 Block Number 610 ...","610 Ang Mo Kio Ave 4, Singapore 560610","(1.3793747, 103.838986, 0.0)",1.379375,103.838986,0.0
3,"Ang Mo Kio, Ang Mo Kio Ave 10 Block Number 474...","474 Ang Mo Kio Ave 10, Singapore 560474","(1.362884, 103.8575989, 0.0)",1.362884,103.857599,0.0
4,"Ang Mo Kio, Ang Mo Kio Ave 5 Block Number 604 ...","604 Ang Mo Kio Ave 5, Singapore 560604","(1.379907, 103.835824, 0.0)",1.379907,103.835824,0.0
...,...,...,...,...,...,...
9084,"Choa Chu Kang, Choa Chu Kang Ave 1 Block Numbe...","Blk, 809A Choa Chu Kang Ave 1, Singapore 681809","(1.375306, 103.7448487, 0.0)",1.375306,103.744849,0.0
9099,"Choa Chu Kang, Choa Chu Kang Ave 1 Block Numbe...","807C Choa Chu Kang Ave 1, Singapore 683807","(1.376412, 103.7455959, 0.0)",1.376412,103.745596,0.0
9100,"Clementi, Clementi Ave 3 Block Number 440A Sin...","Clementi Ave 3, Singapore","(1.3158746, 103.7637147, 0.0)",1.315875,103.763715,0.0
9118,"Choa Chu Kang, Choa Chu Kang Ave 1 Block Numbe...","Choa Chu Kang Ave 1, Singapore 681808","(1.375781, 103.7454887, 0.0)",1.375781,103.745489,0.0
